In [1]:
import pandas as pd
import os

## Percentages

In [75]:
train_df_path ="/PATH TO DATASET CSV FILES IN YOUR SERVER/split/new_train.csv"
test_df_path ="/PATH TO DATASET CSV FILES IN YOUR SERVER/split/new_test.csv"
val_df_path = "/PATH TO DATASET CSV FILES IN YOUR SERVER/split/new_valid.csv"
val_df = pd.read_csv(val_df_path)
train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [76]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

298137
37300
36421


In [3]:
WholeData = test_df.append([val_df, train_df])


In [5]:
print("Number of images:",len(WholeData) )
Whole =WholeData.groupby("subject_id")
print("Number of unique patients:",len(Whole.count()) )


Number of images: 371858
Number of unique patients: 65079


In [10]:
def preprocess_MIMIC(split):
    
    details = pd.read_csv("/scratch/gobi2/projects/ml4h/datasets/new_split/mimic-cxr-metadata-detail.csv")
    details = details.drop(columns=['dicom_id', 'study_id', 'religion', 'marital_status', 'gender'])
    details.drop_duplicates(subset="subject_id", keep="first", inplace=True)
    df = pd.merge(split, details)
    
    copy_sunbjectid = df['subject_id'] 
    df.drop(columns = ['subject_id'])
    
    df = df.replace(
            [[None], -1, "[False]", "[True]", "[ True]", 'UNABLE TO OBTAIN', 'UNKNOWN', 'MARRIED', 'LIFE PARTNER',
             'DIVORCED', 'SEPARATED', '0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90',
             '>=90'],
            [0, 0, 0, 1, 1, 0, 0, 'MARRIED/LIFE PARTNER', 'MARRIED/LIFE PARTNER', 'DIVORCED/SEPARATED',
             'DIVORCED/SEPARATED', '0-20', '0-20', '20-40', '20-40', '40-60', '40-60', '60-80', '60-80', '80-', '80-'])
    
    df['subject_id'] = copy_sunbjectid
    df['Age'] = df["age_decile"]
    df['Sex'] = df["gender"]
    df = df.drop(columns=["age_decile", 'gender'])

    
    return df

In [11]:
test_df = preprocess_MIMIC(test_df)
train_df = preprocess_MIMIC(train_df)
val_df = preprocess_MIMIC(val_df)
WholeData = test_df.append([val_df, train_df])

In [12]:
WholeDataX =WholeData.groupby("Sex")
WholeDataX_df = WholeDataX.describe()
WholeDataX_df

subject_id                                                      \
         count          mean           std         min         25%   
Sex                                                                  
F     177853.0  1.499090e+07  2.883554e+06  10000032.0  12489621.0   
M     194005.0  1.501641e+07  2.879538e+06  10000764.0  12493783.0   

                                        No Finding            ... Fracture  \
            50%         75%         max      count      mean  ...      75%   
Sex                                                           ...            
F    14985054.0  17503573.0  19999987.0   177853.0  0.409585  ...      0.0   
M    15053729.0  17487765.0  19999442.0   194005.0  0.354290  ...      0.0   

         Support Devices                                               
     max           count      mean       std  min  25%  50%  75%  max  
Sex                                                                    
F    1.0        177853.0  0.193176  0.394791  0.0  0.0  0.0  0.0  1.0  
M    1.0        194005.0  0.253617  0.435082  0.0  0.0  0.0  1.0  1.0  

[2 rows x 120 columns]

In [13]:
dfWhole_Sex = WholeDataX_df["subject_id"]['count']
total_CXP= dfWhole_Sex["F"]+dfWhole_Sex["M"]
Male_percent = 100*dfWhole_Sex["M"]/total_CXP
print("Male Percent:  ",Male_percent)

print("female Percent:  ",100*dfWhole_Sex["F"]/total_CXP)
print("#images",total_CXP) 

Male Percent:   52.17179676112925
female Percent:   47.82820323887075
#images 371858.0


In [14]:
WholeData_Age = WholeData[WholeData.Age != 0]
WholeDataAge =WholeData_Age.groupby("Age")
WholeDataAge_df = WholeDataAge.describe()
WholeDataAge_df

subject_id                                                      \
           count          mean           std         min         25%   
Age                                                                    
0-20      8181.0  1.508950e+07  2.894724e+06  10002157.0  12641071.0   
20-40    72516.0  1.502293e+07  2.891342e+06  10001038.0  12532170.0   
40-60   138247.0  1.497541e+07  2.865755e+06  10000032.0  12473093.0   
60-80   126799.0  1.501545e+07  2.894507e+06  10000898.0  12457153.0   
80-      25877.0  1.501857e+07  2.869053e+06  10000764.0  12591968.0   

                                          No Finding            ... Fracture  \
              50%         75%         max      count      mean  ...      75%   
Age                                                             ...            
0-20   15172839.0  17632697.0  19999733.0     8181.0  0.705293  ...      0.0   
20-40  14993854.0  17518261.5  19999270.0    72516.0  0.580065  ...      0.0   
40-60  14976423.0  17421577.0  19999987.0   138247.0  0.403379  ...      0.0   
60-80  15056964.0  17536303.0  19999287.0   126799.0  0.261524  ...      0.0   
80-    15093387.0  17530381.0  19998497.0    25877.0  0.183329  ...      0.0   

           Support Devices                                               
       max           count      mean       std  min  25%  50%  75%  max  
Age                                                                      
0-20   1.0          8181.0  0.089353  0.285271  0.0  0.0  0.0  0.0  1.0  
20-40  1.0         72516.0  0.157703  0.364465  0.0  0.0  0.0  0.0  1.0  
40-60  1.0        138247.0  0.218826  0.413452  0.0  0.0  0.0  0.0  1.0  
60-80  1.0        126799.0  0.267636  0.442729  0.0  0.0  0.0  1.0  1.0  
80-    1.0         25877.0  0.275843  0.446947  0.0  0.0  0.0  1.0  1.0  

[5 rows x 120 columns]

In [15]:
dfWhole_Age1 = WholeDataAge_df["subject_id"]['count']
total_CXR = WholeDataAge_df["subject_id"]['count'].sum()
Age1 = round(100*(dfWhole_Age1["0-20"])/total_CXR,2)
Age2 = round(100*(dfWhole_Age1["20-40"])/total_CXR,2)
Age3 = round(100*(dfWhole_Age1["40-60"])/total_CXR,2)
Age4 = round(100*(dfWhole_Age1["60-80"])/total_CXR,2)
Age5 = round(100*(dfWhole_Age1["80-"])/total_CXR,2)
print("0-20 Percent:  ",Age1)
print("20-40 Percent:  ",Age2)
print("40-60 Percent:  ",Age3)
print("60-80 Percent:  ",Age4)
print("80- Percent:  ",Age5)
2.2+ 19.51+37.20+34.12+6.96

0-20 Percent:   2.2
20-40 Percent:   19.51
40-60 Percent:   37.2
60-80 Percent:   34.12
80- Percent:   6.96


99.99

In [16]:
2.2+ 19.51+37.20+34.12+6.96

99.99

In [17]:
WholeData_ins = WholeData[WholeData.insurance != 0]
WholeDataI =WholeData_ins.groupby("insurance")
WholeDataI_df = WholeDataI.describe()
total_CXR = WholeDataI_df["subject_id"]['count'].sum()
dfWhole_Ins = WholeDataI_df["subject_id"]['count']
Ins1 = 100*(dfWhole_Ins["Medicare"])/total_CXR
Ins2 = 100*(dfWhole_Ins["Medicaid"])/total_CXR
Ins3 = 100*(dfWhole_Ins["Other"])/total_CXR

print("Medicare Percent:  ",Ins1)
print("Medicaid Percent:  ",Ins2)
print("Other Percent:  ",Ins3)

46.07 + 8.98 + 44.95

Medicare Percent:   46.07218293015006
Medicaid Percent:   8.977604745468328
Other Percent:   44.95021232438161


100.0

In [18]:
WholeData_Rac = WholeData[WholeData.race != 0]
WholeDataR =WholeData_Rac.groupby("race")
WholeDataR_df = WholeDataR.describe()
WholeDataR_df

subject_id                              \
                                   count          mean           std   
race                                                                   
AMERICAN INDIAN/ALASKA NATIVE      761.0  1.501666e+07  2.823667e+06   
ASIAN                             8479.0  1.508827e+07  2.851218e+06   
BLACK/AFRICAN AMERICAN           48595.0  1.497838e+07  2.896457e+06   
HISPANIC/LATINO                  16760.0  1.498038e+07  2.887127e+06   
OTHER                            10010.0  1.508103e+07  2.991056e+06   
WHITE                           176837.0  1.501440e+07  2.877738e+06   

                                                                               \
                                      min         25%         50%         75%   
race                                                                            
AMERICAN INDIAN/ALASKA NATIVE  10248673.0  12493668.0  14382048.0  18116591.0   
ASIAN                          10005329.0  12567919.0  15206209.0  17651038.0   
BLACK/AFRICAN AMERICAN         10000935.0  12468956.0  14936098.0  17479533.0   
HISPANIC/LATINO                10011607.0  12373624.0  15056964.0  17421215.0   
OTHER                          10003637.0  12408654.0  15014371.0  17792682.0   
WHITE                          10000032.0  12515419.0  15067471.0  17481338.0   

                                          No Finding            ... Fracture  \
                                      max      count      mean  ...      75%   
race                                                            ...            
AMERICAN INDIAN/ALASKA NATIVE  19998198.0      761.0  0.314060  ...      0.0   
ASIAN                          19990563.0     8479.0  0.326454  ...      0.0   
BLACK/AFRICAN AMERICAN         19999287.0    48595.0  0.413787  ...      0.0   
HISPANIC/LATINO                19983948.0    16760.0  0.451014  ...      0.0   
OTHER                          19979173.0    10010.0  0.329870  ...      0.0   
WHITE                          19999733.0   176837.0  0.305954  ...      0.0   

                                   Support Devices                           \
                               max           count      mean       std  min   
race                                                                          
AMERICAN INDIAN/ALASKA NATIVE  1.0           761.0  0.392904  0.488717  0.0   
ASIAN                          1.0          8479.0  0.286472  0.452139  0.0   
BLACK/AFRICAN AMERICAN         1.0         48595.0  0.182426  0.386199  0.0   
HISPANIC/LATINO                1.0         16760.0  0.186277  0.389342  0.0   
OTHER                          1.0         10010.0  0.307592  0.461520  0.0   
WHITE                          1.0        176837.0  0.281219  0.449596  0.0   

                                                   
                               25%  50%  75%  max  
race                                               
AMERICAN INDIAN/ALASKA NATIVE  0.0  0.0  1.0  1.0  
ASIAN                          0.0  0.0  1.0  1.0  
BLACK/AFRICAN AMERICAN         0.0  0.0  0.0  1.0  
HISPANIC/LATINO                0.0  0.0  0.0  1.0  
OTHER                          0.0  0.0  1.0  1.0  
WHITE                          0.0  0.0  1.0  1.0  

[6 rows x 120 columns]

In [19]:
WholeData_Rac = WholeData[WholeData.race != 0]
WholeDataR =WholeData_Rac.groupby("race")
WholeDataR_df = WholeDataR.describe()
total_CXR = WholeDataR_df["subject_id"]['count'].sum()
dfWhole_Race = WholeDataR_df["subject_id"]['count']
Rac1 = 100*(dfWhole_Race["WHITE"])/total_CXR
Rac2 = 100*(dfWhole_Race["BLACK/AFRICAN AMERICAN"])/total_CXR
Rac3 = 100*(dfWhole_Race["OTHER"])/total_CXR
Rac4 = 100*(dfWhole_Race["ASIAN"])/total_CXR
Rac5 = 100*(dfWhole_Race["HISPANIC/LATINO"])/total_CXR
Rac6 = 100*(dfWhole_Race["AMERICAN INDIAN/ALASKA NATIVE"])/total_CXR

print("WHITE Percent:  ",round( Rac1,2))
print("BLACK/AFRICAN AMERICAN Percent:  ",round(Rac2,2))
print("OTHER Percent:  ",round(Rac3,2))
print("ASIAN Percent:  ",round(Rac4,2))
print("HISPANIC/LATINO Percent:  ",round(Rac5,2))
print("AMERICAN INDIAN/ALASKA NATIVE Percent:  ",round(Rac6,2))
print("Unkhonwn or unable to obtain Percent:  ",round(100- (Rac1 + Rac2 + Rac3 +Rac4 + Rac5 + Rac6),2))

Rac1 + Rac2 + Rac3 +Rac4 + Rac5 + Rac6

WHITE Percent:   67.64
BLACK/AFRICAN AMERICAN Percent:   18.59
OTHER Percent:   3.83
ASIAN Percent:   3.24
HISPANIC/LATINO Percent:   6.41
AMERICAN INDIAN/ALASKA NATIVE Percent:   0.29
Unkhonwn or unable to obtain Percent:   0.0


99.99999999999999

## Subgroup-specific underdiagnosis rate

### Sex

In [20]:
FP5_sex = pd.read_csv("./results77/FPR_FNR_NF_sex.csv")
FP4_sex = pd.read_csv("./results19/FPR_FNR_NF_sex.csv")
FP3_sex = pd.read_csv("./results38/FPR_FNR_NF_sex.csv")
FP2_sex = pd.read_csv("./results47/FPR_FNR_NF_sex.csv")
FP1_sex = pd.read_csv("./results31/FPR_FNR_NF_sex.csv")
FP_sex = FP1_sex.append([FP2_sex, FP3_sex,FP4_sex, FP5_sex])
F_sex_df = FP_sex.describe()

In [21]:
print("FPR and FNR distribiution in sexes")
print(round(F_sex_df.loc['mean'],3))

FPR and FNR distribiution in sexes
Unnamed: 0        0.000
#M            12094.000
FPR_M             0.219
FNR_M             0.198
#F            10420.000
FPR_F             0.250
FNR_F             0.155
Name: mean, dtype: float64


In [22]:
print("FPR and FNR distribiution in sexes Confidence interval")
round(1.96 * F_sex_df.loc['std'] / np.sqrt(5),3)

FPR and FNR distribiution in sexes Confidence interval


Unnamed: 0    0.000
#M            0.000
FPR_M         0.007
FNR_M         0.007
#F            0.000
FPR_F         0.007
FNR_F         0.005
Name: std, dtype: float64

In [23]:
print("MIMIC: Mean FPR distribiution over sexes")
mean([F_sex_df.loc['mean']["FPR_F"], F_sex_df.loc['mean']["FPR_M"]])

MIMIC: Mean FPR distribiution over sexes


0.2343

In [24]:
Sex = ['M','F' ]
FPR_FNR_sex_df = pd.DataFrame(Sex, columns=["sex"])

def FiveRunSubgroup(factors, df_in, df_out):
    FPR = []
    FNR = []
    Percent = []
    CI_FPR =[]
    CI_FNR =[]
    ConfI = 1.96 * df_in.loc['std'] / np.sqrt(5)
    
    for fact in factors:
        Percent.append(round(df_in.loc['mean']['#'+fact],3))
        FPR.append(round(df_in.loc['mean']['FPR_'+fact],3))
        FNR.append(round(df_in.loc['mean']['FNR_'+fact],3))
        CI_FPR.append(round(ConfI.loc['FPR_'+fact],3))
        CI_FNR.append(round(ConfI.loc['FNR_'+fact],3))     
        
    df_out['#'] = Percent    
    df_out['FPR']=FPR 
    df_out['CI_FPR']=CI_FPR  

    df_out['FNR']=FNR 
    df_out['CI_FNR']=CI_FNR

    return df_out

FPR_FNR_sex_df=FiveRunSubgroup(Sex, F_sex_df, FPR_FNR_sex_df)
FPR_FNR_sex_df.to_csv('./results/Subgroun_FNR_FPR_Sex.csv')
FPR_FNR_sex_df

,sex,#,FPR,CI_FPR,FNR,CI_FNR
0,M,12094.0,0.219,0.007,0.198,0.007
1,F,10420.0,0.250,0.007,0.155,0.005


### Age

In [25]:
FP5_age = pd.read_csv("./results77/FPR_FNR_NF_age.csv")
FP4_age = pd.read_csv("./results19/FPR_FNR_NF_age.csv")
FP3_age = pd.read_csv("./results38/FPR_FNR_NF_age.csv")
FP2_age = pd.read_csv("./results47/FPR_FNR_NF_age.csv")
FP1_age = pd.read_csv("./results31/FPR_FNR_NF_age.csv")
FP_age = FP1_age.append([FP2_age, FP3_age,FP4_age, FP5_age])
F_age_df = FP_age.describe()


In [26]:
print("FPR distribiution in ages")
print(round(F_age_df.loc['mean'],3))

FPR distribiution in ages
Unnamed: 0       0.000
#60-80        9006.000
FPR_60-80        0.163
FNR_60-80        0.328
#40-60        8187.000
FPR_40-60        0.274
FNR_40-60        0.163
#20-40        3060.000
FPR_20-40        0.390
FNR_20-40        0.064
#80-          2016.000
FPR_80-          0.114
FNR_80-          0.420
#0-20          245.000
FPR_0-20         0.486
FNR_0-20         0.028
Name: mean, dtype: float64


In [27]:
print("MIMIC: Mean FPR distribiution over ages")
mean([F_age_df.loc['mean']["FPR_60-80"], F_age_df.loc['mean']["FPR_40-60"],
       F_age_df.loc['mean']["FPR_20-40"], F_age_df.loc['mean']["FPR_0-20"],
       F_age_df.loc['mean']["FPR_80-"]])

MIMIC: Mean FPR distribiution over ages


0.28532

In [28]:
round(1.96 * F_age_df.loc['std'] / np.sqrt(5),3)

Unnamed: 0    0.000
#60-80        0.000
FPR_60-80     0.007
FNR_60-80     0.012
#40-60        0.000
FPR_40-60     0.007
FNR_40-60     0.006
#20-40        0.000
FPR_20-40     0.010
FNR_20-40     0.003
#80-          0.000
FPR_80-       0.007
FNR_80-       0.020
#0-20         0.000
FPR_0-20      0.024
FNR_0-20      0.007
Name: std, dtype: float64

In [29]:
Age = ['0-20','20-40','40-60','60-80','80-' ]
FPR_FNR_Age_df = pd.DataFrame(Age, columns=["Age"])

FPR_FNR_Age_df=FiveRunSubgroup(Age, F_age_df, FPR_FNR_Age_df)
FPR_FNR_Age_df.to_csv('./results/Subgroun_FNR_FPR_Age.csv')
FPR_FNR_Age_df

,Age,#,FPR,CI_FPR,FNR,CI_FNR
0,0-20,245.0,0.486,0.024,0.028,0.007
1,20-40,3060.0,0.390,0.010,0.064,0.003
2,40-60,8187.0,0.274,0.007,0.163,0.006
3,60-80,9006.0,0.163,0.007,0.328,0.012
4,80-,2016.0,0.114,0.007,0.420,0.020


### Race

In [30]:
FP5_race = pd.read_csv("./results77/FPR_FNR_NF_race.csv")
FP4_race = pd.read_csv("./results19/FPR_FNR_NF_race.csv")
FP3_race = pd.read_csv("./results38/FPR_FNR_NF_race.csv")
FP2_race = pd.read_csv("./results47/FPR_FNR_NF_race.csv")
FP1_race = pd.read_csv("./results31/FPR_FNR_NF_race.csv")

FP_race = FP1_race.append([FP2_race, FP3_race,FP4_race, FP5_race])
F_race_df = FP_race.describe()

In [31]:
print("FPR distribiution in races")
print(round(F_race_df.loc['mean'],3))

FPR distribiution in races
Unnamed: 0          0.000
#White          11620.000
FPR_White           0.170
FNR_White           0.253
#Black           2924.000
FPR_Black           0.276
FNR_Black           0.168
#Hisp             998.000
FPR_Hisp            0.275
FNR_Hisp            0.162
#Other            725.000
FPR_Other           0.176
FNR_Other           0.201
#Asian            407.000
FPR_Asian           0.217
FNR_Asian           0.134
#American          41.000
FPR_American        0.176
FNR_American        0.246
Name: mean, dtype: float64


In [32]:
round(1.96 * F_race_df.loc['std'] / np.sqrt(5),3)

Unnamed: 0      0.000
#White          0.000
FPR_White       0.006
FNR_White       0.006
#Black          0.000
FPR_Black       0.006
FNR_Black       0.008
#Hisp           0.000
FPR_Hisp        0.012
FNR_Hisp        0.012
#Other          0.000
FPR_Other       0.007
FNR_Other       0.011
#Asian          0.000
FPR_Asian       0.007
FNR_Asian       0.007
#American       0.000
FPR_American    0.032
FNR_American    0.030
Name: std, dtype: float64

In [33]:
Race = ['White','Black','Hisp','Other','Asian','American' ]
FPR_FNR_Race_df = pd.DataFrame(Race, columns=["Race"])

FPR_FNR_Race_df=FiveRunSubgroup(Race, F_race_df, FPR_FNR_Race_df)
FPR_FNR_Race_df.to_csv('./results/Subgroun_FNR_FPR_Race.csv')
FPR_FNR_Race_df

,Race,#,FPR,CI_FPR,FNR,CI_FNR
0,White,11620.0,0.170,0.006,0.253,0.006
1,Black,2924.0,0.276,0.006,0.168,0.008
2,Hisp,998.0,0.275,0.012,0.162,0.012
3,Other,725.0,0.176,0.007,0.201,0.011
4,Asian,407.0,0.217,0.007,0.134,0.007
5,American,41.0,0.176,0.032,0.246,0.030


## Insurance

In [34]:
FP5_insurance = pd.read_csv("./results77/FPR_FNR_NF_insurance.csv")
FP4_insurance = pd.read_csv("./results19/FPR_FNR_NF_insurance.csv")
FP3_insurance = pd.read_csv("./results38/FPR_FNR_NF_insurance.csv")
FP2_insurance = pd.read_csv("./results47/FPR_FNR_NF_insurance.csv")
FP1_insurance = pd.read_csv("./results31/FPR_FNR_NF_insurance.csv")
FP_insurance = FP1_insurance.append([FP2_insurance, FP3_insurance,FP4_insurance, FP5_insurance])
FP_insurance_df = FP_insurance.describe()

In [35]:
print("FPR distribiution in insurances")
print(round(FP_insurance_df.loc['mean'],3))

FPR distribiution in insurances
Unnamed: 0         0.000
#Medicare       8507.000
FPR_Medicare       0.162
FNR_Medicare       0.310
#Other          7369.000
FPR_Other          0.208
FNR_Other          0.178
#Medicaid       1591.000
FPR_Medicaid       0.274
FNR_Medicaid       0.166
Name: mean, dtype: float64


In [36]:
round(1.96 * FP_insurance_df.loc['std'] / np.sqrt(5),3)

Unnamed: 0      0.000
#Medicare       0.000
FPR_Medicare    0.007
FNR_Medicare    0.007
#Other          0.000
FPR_Other       0.005
FNR_Other       0.005
#Medicaid       0.000
FPR_Medicaid    0.011
FNR_Medicaid    0.010
Name: std, dtype: float64

In [37]:
ConfI = 1.96 * FP_insurance_df.loc['std'] / np.sqrt(5)
ConfI.loc['FPR_Medicare']

0.006692761462953838

In [38]:
Insurance = ['Medicare','Other','Medicaid' ]

FPR_FNR_Ins_df = pd.DataFrame(Insurance, columns=["Insurance"])

FP_insurance_df=FiveRunSubgroup(Insurance, FP_insurance_df, FPR_FNR_Ins_df)
FP_insurance_df.to_csv('./results/Subgroun_FNR_FPR_Insurance.csv')
FP_insurance_df

,Insurance,#,FPR,CI_FPR,FNR,CI_FNR
0,Medicare,8507.0,0.162,0.007,0.310,0.007
1,Other,7369.0,0.208,0.005,0.178,0.005
2,Medicaid,1591.0,0.274,0.011,0.166,0.010


## Intersectional identity

### Age-Sex

In [39]:
FP5_agesex = pd.read_csv("./results77/FP_AgeSex.csv")
FP4_agesex= pd.read_csv("./results19/FP_AgeSex.csv")
FP3_agesex = pd.read_csv("./results38/FP_AgeSex.csv")
FP2_agesex = pd.read_csv("./results47/FP_AgeSex.csv")
FP1_agesex = pd.read_csv("./results31/FP_AgeSex.csv")
FP_agesex = FP1_agesex.append([FP2_agesex, FP3_agesex,FP4_agesex, FP5_agesex])
FP_AgeSex =FP_agesex.groupby("Age")   
FP_AgSx_df = FP_AgeSex.describe()

In [40]:
FP_AgSx_df

Unnamed: 0                                        M          ...         \
           count mean  std  min  25%  50%  75%  max count    mean  ...    75%   
Age                                                                ...          
0-20         5.0  4.0  0.0  4.0  4.0  4.0  4.0  4.0   5.0  0.4298  ...  0.449   
20-40        5.0  2.0  0.0  2.0  2.0  2.0  2.0  2.0   5.0  0.3078  ...  0.312   
40-60        5.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0   5.0  0.2636  ...  0.268   
60-80        5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   5.0  0.1590  ...  0.165   
80-          5.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0   5.0  0.1030  ...  0.107   

                 F                                                       
         max count    mean       std    min    25%    50%    75%    max  
Age                                                                      
0-20   0.462   5.0  0.5842  0.026224  0.562  0.573  0.573  0.584  0.629  
20-40  0.332   5.0  0.5140  0.006782  0.507  0.511  0.512  0.515  0.525  
40-60  0.271   5.0  0.2852  0.008701  0.270  0.286  0.289  0.290  0.291  
60-80  0.166   5.0  0.1674  0.009127  0.153  0.164  0.171  0.174  0.175  
80-    0.108   5.0  0.1224  0.011283  0.108  0.116  0.121  0.131  0.136  

[5 rows x 24 columns]

In [41]:
print("Age-Sex over male")
round(FP_AgSx_df['M']['mean'],3)

Age-Sex over male


Age
0-20     0.430
20-40    0.308
40-60    0.264
60-80    0.159
80-      0.103
Name: mean, dtype: float64

In [42]:
round(1.96 * FP_AgSx_df['M']["std"] / np.sqrt(5),3)

Age
0-20     0.032
20-40    0.014
40-60    0.006
60-80    0.006
80-      0.005
Name: std, dtype: float64

In [43]:
print("mean: ", round(FP_AgSx_df['M']['mean'],3).mean() )

mean:  0.2528


In [44]:
round(FP_AgSx_df['F']['mean'],3)

Age
0-20     0.584
20-40    0.514
40-60    0.285
60-80    0.167
80-      0.122
Name: mean, dtype: float64

In [45]:
round(1.96 * FP_AgSx_df['F']["std"] / np.sqrt(5),3)

Age
0-20     0.023
20-40    0.006
40-60    0.008
60-80    0.008
80-      0.010
Name: std, dtype: float64

In [46]:
CI = round(1.96 * FP_AgSx_df['F']["std"] / np.sqrt(5),3)

In [47]:
factors = ['F', 'M']
age =['0-20', '20-40', '40-60', '60-80','80-']
AgeSex_df = pd.DataFrame(age, columns=["Age"])
Num_df =  pd.read_csv("./results77/Num_AgeSex.csv")


def FiveRun(factors,want_df,df,Num):
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])
        
        Num['Num_'+factor] = Num[factor]
    
    Num = Num.drop(factors,axis=1)
    want_df = want_df.merge(Num,left_on="Age", right_on="Age")
    
    return want_df

want = FiveRun(factors,AgeSex_df,FP_AgSx_df,Num_df)
want.to_csv('./results/Inter_AgeSex.csv')
want

,Age,F,CI_F,M,CI_M,Unnamed: 0,Num_F,Num_M
0,0-20,0.584,0.023,0.430,0.032,4,89,156
1,20-40,0.514,0.006,0.308,0.014,2,1222,1838
2,40-60,0.285,0.008,0.264,0.006,1,3816,4371
3,60-80,0.167,0.008,0.159,0.006,0,4130,4876
4,80-,0.122,0.010,0.103,0.005,3,1163,853


In [48]:
FP_NF_5_agesex = pd.read_csv("./results77/FP_FN_AgeSex.csv")
FP_NF_4_agesex= pd.read_csv("./results19/FP_FN_AgeSex.csv")
FP_NF_3_agesex = pd.read_csv("./results38/FP_FN_AgeSex.csv")
FP_NF_2_agesex = pd.read_csv("./results47/FP_FN_AgeSex.csv")
FP_NF_1_agesex = pd.read_csv("./results31/FP_FN_AgeSex.csv")
FP_NF_agesex = FP_NF_1_agesex.append([FP_NF_2_agesex, FP_NF_3_agesex,FP_NF_4_agesex, FP_NF_5_agesex])
FP_NF_AgeSex =FP_NF_agesex.groupby("Age")   
FP_NF_AgSx_df = FP_NF_AgeSex.describe()

In [49]:
factors = ['FPR_M','FNR_M','FPR_F','FNR_F']
age =['0-20', '20-40', '40-60', '60-80','80-']
AgeSex_df = pd.DataFrame(age, columns=["Age"])


def FiveRun_FP_NF(factors,want_df,df):
    
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])
    
    return want_df

want = FiveRun_FP_NF(factors, AgeSex_df, FP_NF_AgSx_df)
want.to_csv('./results/Inter_AgeSex_FNR_FPR.csv')

want

,Age,FPR_M,CI_FPR_M,FNR_M,CI_FNR_M,FPR_F,CI_FPR_F,FNR_F,CI_FNR_F
0,0-20,0.430,0.032,0.032,0.009,0.584,0.023,0.026,0.007
1,20-40,0.308,0.014,0.099,0.006,0.514,0.006,0.037,0.002
2,40-60,0.264,0.006,0.171,0.008,0.285,0.008,0.157,0.006
3,60-80,0.159,0.006,0.338,0.013,0.167,0.008,0.317,0.013
4,80-,0.103,0.005,0.416,0.012,0.122,0.010,0.422,0.025


In [8]:
# code in fase_pos.py
print('Number of images in age sex intersection with actual NF=1')
pd.read_csv('./results47/Num_AgeSex_NF1.csv')

Number of images in age sex intersection with actual NF=1


,Unnamed: 0,Age,M,F
0,0,60-80,1691,1526
1,1,40-60,2659,2770
2,2,20-40,1828,2374
3,3,80-,185,316
4,4,0-20,155,397


In [7]:
print('Number of images in age sex intersection with actual NF=0')
pd.read_csv('./results47/Num_AgeSex.csv')

Number of images in age sex intersection with actual NF=0


,Unnamed: 0,Age,M,F
0,0,60-80,4876,4130
1,1,40-60,4371,3816
2,2,20-40,1838,1222
3,3,80-,853,1163
4,4,0-20,156,89


### Insurace -Sex

In [50]:
FP5_InsSex = pd.read_csv("./results77/FP_InsSex.csv")
FP4_InsSex = pd.read_csv("./results19/FP_InsSex.csv")
FP3_InsSex = pd.read_csv("./results38/FP_InsSex.csv")
FP2_InsSex = pd.read_csv("./results47/FP_InsSex.csv")
FP1_InsSex = pd.read_csv("./results31/FP_InsSex.csv")
FP_InsSex = FP1_InsSex.append([FP2_InsSex, FP3_InsSex,FP4_InsSex, FP5_InsSex])
FP_InSx =FP_InsSex.groupby("Insurance")
FP_InSx_df = FP_InSx.describe()

In [51]:
round(FP_InSx_df['M']['mean'],3)

Insurance
Medicaid    0.268
Medicare    0.157
Other       0.188
Name: mean, dtype: float64

In [52]:
round(1.96 * FP_InSx_df['M']["std"] / np.sqrt(5),3)

Insurance
Medicaid    0.006
Medicare    0.007
Other       0.006
Name: std, dtype: float64

In [53]:
round(FP_InSx_df['F']['mean'],3)

Insurance
Medicaid    0.280
Medicare    0.168
Other       0.235
Name: mean, dtype: float64

In [54]:
CI=round(1.96 * FP_InSx_df['F']["std"] / np.sqrt(5),3)
CI.loc['Medicaid']

0.015

In [55]:
round(1.96 * FP_InSx_df['F']["std"] / np.sqrt(5),3)

Insurance
Medicaid    0.015
Medicare    0.010
Other       0.005
Name: std, dtype: float64

In [56]:
factors = ['F', 'M']
Insurance = ['Medicaid','Medicare','Other']
SexIns_df = pd.DataFrame(Insurance, columns=["Insurance"])
Num_df =  pd.read_csv("./results77/Num_InsSex.csv")


def FiveRun(factors,want_df,df,Num):
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])        
        Num['Num_'+factor] = Num[factor]
    
    Num = Num.drop(factors,axis=1)
    want_df = want_df.merge(Num,left_on="Insurance", right_on="Insurance")

    return want_df

want = FiveRun(factors,SexIns_df,FP_InSx_df,Num_df)
want.to_csv('./results/Inter_SexIns.csv')
want

,Insurance,F,CI_F,M,CI_M,Unnamed: 0,Num_F,Num_M
0,Medicaid,0.280,0.015,0.268,0.006,2,834,757
1,Medicare,0.168,0.010,0.157,0.007,0,3981,4526
2,Other,0.235,0.005,0.188,0.006,1,3133,4232


In [57]:
FP5_InsSex = pd.read_csv("./results77/FP_FN_InsSex.csv")
FP4_InsSex = pd.read_csv("./results19/FP_FN_InsSex.csv")
FP3_InsSex = pd.read_csv("./results38/FP_FN_InsSex.csv")
FP2_InsSex = pd.read_csv("./results47/FP_FN_InsSex.csv")
FP1_InsSex = pd.read_csv("./results31/FP_FN_InsSex.csv")
FP_InsSex = FP1_InsSex.append([FP2_InsSex, FP3_InsSex,FP4_InsSex, FP5_InsSex])
FP_InSx =FP_InsSex.groupby("Insurance")
FP_InSx_df = FP_InSx.describe()

In [58]:
factors = ['FPR_M','FNR_M','FPR_F','FNR_F']
Insurance = ['Medicaid','Medicare','Other']
SexIns_df = pd.DataFrame(Insurance, columns=["Insurance"])

want = FiveRun_FP_NF(factors, SexIns_df, FP_InSx_df)
want.to_csv('./results/Inter_SexIns_FNR_FPR.csv')
want

,Insurance,FPR_M,CI_FPR_M,FNR_M,CI_FNR_M,FPR_F,CI_FPR_F,FNR_F,CI_FNR_F
0,Medicaid,0.268,0.006,0.215,0.014,0.280,0.015,0.131,0.010
1,Medicare,0.157,0.007,0.322,0.011,0.168,0.010,0.297,0.012
2,Other,0.188,0.006,0.212,0.005,0.235,0.005,0.148,0.006


In [23]:
print('Number of images in age sex intersection with actual NF=0')
pd.read_csv('./results47/Num_InsSex.csv')

Number of images in age sex intersection with actual NF=0


,Unnamed: 0,Insurance,M,F
0,0,Medicare,4526,3981
1,1,Other,4232,3133
2,2,Medicaid,757,834


In [24]:
print('Number of images in Ins sex intersection with actual NF=1')
pd.read_csv('./results47/Num_InsSex_NF1.csv')

Number of images in age sex intersection with actual NF=0


,Unnamed: 0,Insurance,M,F
0,0,Medicare,1556,1574
1,1,Other,2097,2440
2,2,Medicaid,473,642


## Insurance-Race

In [59]:
FP5_InsRace = pd.read_csv("./results77/FP_InsRace.csv")
FP4_InsRace = pd.read_csv("./results19/FP_InsRace.csv")
FP3_InsRace = pd.read_csv("./results38/FP_InsRace.csv")
FP2_InsRace = pd.read_csv("./results47/FP_InsRace.csv")
FP1_InsRace = pd.read_csv("./results31/FP_InsRace.csv")
FP_InsRace = FP1_InsRace.append([FP2_InsRace, FP3_InsRace,FP4_InsRace, FP5_InsRace])
FP_InsRace =FP_InsRace.groupby("race")
FP_InRa_df =FP_InsRace.describe()

In [60]:
round(FP_InRa_df['Medicare']['mean'],3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.113
ASIAN                            0.121
BLACK/AFRICAN AMERICAN           0.250
HISPANIC/LATINO                  0.292
OTHER                            0.151
WHITE                            0.149
Name: mean, dtype: float64

In [61]:
round(1.96 * FP_InRa_df['Medicare']["std"] / np.sqrt(5),3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.033
ASIAN                            0.012
BLACK/AFRICAN AMERICAN           0.009
HISPANIC/LATINO                  0.009
OTHER                            0.011
WHITE                            0.007
Name: std, dtype: float64

In [62]:
round(FP_InRa_df['Medicaid']['mean'],3)

race
AMERICAN INDIAN/ALASKA NATIVE      NaN
ASIAN                            0.313
BLACK/AFRICAN AMERICAN           0.370
HISPANIC/LATINO                  0.228
OTHER                            0.220
WHITE                            0.232
Name: mean, dtype: float64

In [63]:
round(1.96 * FP_InRa_df['Medicaid']["std"] / np.sqrt(5),3)

race
AMERICAN INDIAN/ALASKA NATIVE      NaN
ASIAN                            0.011
BLACK/AFRICAN AMERICAN           0.015
HISPANIC/LATINO                  0.019
OTHER                            0.016
WHITE                            0.010
Name: std, dtype: float64

In [64]:
round(FP_InRa_df['Other']['mean'],3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.346
ASIAN                            0.232
BLACK/AFRICAN AMERICAN           0.266
HISPANIC/LATINO                  0.296
OTHER                            0.186
WHITE                            0.191
Name: mean, dtype: float64

In [65]:
round(1.96 * FP_InRa_df['Other']["std"] / np.sqrt(5),3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.036
ASIAN                            0.014
BLACK/AFRICAN AMERICAN           0.004
HISPANIC/LATINO                  0.010
OTHER                            0.008
WHITE                            0.006
Name: std, dtype: float64

In [66]:
factors = ['Medicaid', 'Other','Medicare']
race = ['AMERICAN INDIAN/ALASKA NATIVE','ASIAN','BLACK/AFRICAN AMERICAN','HISPANIC/LATINO','OTHER','WHITE']
RaceIns_df = pd.DataFrame(race, columns=["race"])
Num_df =  pd.read_csv("./results77/Num_InsRace.csv")


def FiveRun(factors,want_df,df,Num):
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])        
        Num['Num_'+factor] = Num[factor]
    
    Num = Num.drop(factors,axis=1)
    want_df = want_df.merge(Num,left_on="race", right_on="race")

    return want_df

want = FiveRun(factors,RaceIns_df,FP_InRa_df,Num_df)
want.to_csv('./results/Inter_RaceIns.csv')
want

,race,Medicaid,CI_Medicaid,Other,CI_Other,Medicare,CI_Medicare,Unnamed: 0,Num_Medicaid,Num_Other,Num_Medicare
0,AMERICAN INDIAN/ALASKA NATIVE,NaN,NaN,0.346,0.036,0.113,0.033,5,0,11,30
1,ASIAN,0.313,0.011,0.232,0.014,0.121,0.012,4,90,195,122
2,BLACK/AFRICAN AMERICAN,0.370,0.015,0.266,0.004,0.250,0.009,1,467,1276,1181
3,HISPANIC/LATINO,0.228,0.019,0.296,0.010,0.292,0.009,2,307,461,230
4,OTHER,0.220,0.016,0.186,0.008,0.151,0.011,3,61,397,267
5,WHITE,0.232,0.010,0.191,0.006,0.149,0.007,0,633,4701,6283


In [67]:
FP5_InsRace = pd.read_csv("./results77/FP_FN_InsRace.csv")
FP4_InsRace = pd.read_csv("./results19/FP_FN_InsRace.csv")
FP3_InsRace = pd.read_csv("./results38/FP_FN_InsRace.csv")
FP2_InsRace = pd.read_csv("./results47/FP_FN_InsRace.csv")
FP1_InsRace = pd.read_csv("./results31/FP_FN_InsRace.csv")
FP_InsRace = FP1_InsRace.append([FP2_InsRace, FP3_InsRace,FP4_InsRace, FP5_InsRace])
FP_InsRace =FP_InsRace.groupby("race")
FP_InRa_df =FP_InsRace.describe()

In [68]:
FP_InRa_df.columns

MultiIndex(levels=[['Unnamed: 0', 'FPR_Medicare', 'FNR_Medicare', 'FPR_Other', 'FNR_Other', 'FPR_Medicaid', 'FNR_Medicaid'], ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [69]:
factors = ['FPR_Medicare','FNR_Medicare','FPR_Other','FNR_Other', 'FPR_Medicaid', 'FNR_Medicaid']
race = ['AMERICAN INDIAN/ALASKA NATIVE','ASIAN','BLACK/AFRICAN AMERICAN','HISPANIC/LATINO','OTHER','WHITE']

RaceIns_df = pd.DataFrame(race, columns=["race"])

want = FiveRun_FP_NF(factors, RaceIns_df, FP_InRa_df)
want.to_csv('./results/Inter_RaceIns_FNR_FPR.csv')
want

,race,FPR_Medicare,CI_FPR_Medicare,FNR_Medicare,CI_FNR_Medicare,FPR_Other,CI_FPR_Other,FNR_Other,CI_FNR_Other,FPR_Medicaid,CI_FPR_Medicaid,FNR_Medicaid,CI_FNR_Medicaid
0,AMERICAN INDIAN/ALASKA NATIVE,0.113,0.033,1.000,0.000,0.346,0.036,0.133,0.044,NaN,NaN,0.000,0.000
1,ASIAN,0.121,0.012,0.215,0.018,0.232,0.014,0.119,0.005,0.313,0.011,0.123,0.023
2,BLACK/AFRICAN AMERICAN,0.250,0.009,0.257,0.014,0.266,0.004,0.130,0.005,0.370,0.015,0.128,0.008
3,HISPANIC/LATINO,0.292,0.009,0.190,0.019,0.296,0.010,0.123,0.006,0.228,0.019,0.223,0.025
4,OTHER,0.151,0.011,0.218,0.017,0.186,0.008,0.217,0.006,0.220,0.016,0.105,0.025
5,WHITE,0.149,0.007,0.327,0.006,0.191,0.006,0.202,0.007,0.232,0.010,0.187,0.008


In [37]:
print('Number of images in Ins Rac intersection with actual NF=0')
pd.read_csv('./results47/Num_InsRace.csv')

Number of images in Ins Rac intersection with actual NF=0


,Unnamed: 0,race,Medicare,Other,Medicaid
0,0,WHITE,6283,4701,633
1,1,BLACK/AFRICAN AMERICAN,1181,1276,467
2,2,HISPANIC/LATINO,230,461,307
3,3,OTHER,267,397,61
4,4,ASIAN,122,195,90
5,5,AMERICAN INDIAN/ALASKA NATIVE,30,11,0


In [39]:
print('Number of images in Ins Rac intersection with actual NF=1')
pd.read_csv('./results47/Num_InsRace_NF1.csv')


Number of images in Ins Rac intersection with actual NF=1


,Unnamed: 0,race,Medicare,Other,Medicaid
0,0,WHITE,2181,2548,398
1,1,BLACK/AFRICAN AMERICAN,614,1079,375
2,2,HISPANIC/LATINO,126,435,221
3,3,OTHER,97,205,53
4,4,ASIAN,41,174,52
5,5,AMERICAN INDIAN/ALASKA NATIVE,2,9,2


### Race- Sex

In [70]:
FP5_RaceSex = pd.read_csv("./results77/FP_RaceSex.csv")
FP4_RaceSex = pd.read_csv("./results19/FP_RaceSex.csv")
FP3_RaceSex = pd.read_csv("./results38/FP_RaceSex.csv")
FP2_RaceSex = pd.read_csv("./results47/FP_RaceSex.csv")
FP1_RaceSex = pd.read_csv("./results31/FP_RaceSex.csv")
FP_IRaceSex = FP1_RaceSex.append([FP2_RaceSex, FP3_RaceSex,FP4_RaceSex, FP5_RaceSex])
FP_RaceSex =FP_IRaceSex.groupby("race")
FP_RaceSex_df =FP_RaceSex.describe()

In [71]:
round(FP_RaceSex_df['M']['mean'],3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.165
ASIAN                            0.149
BLACK/AFRICAN AMERICAN           0.256
HISPANIC/LATINO                  0.206
OTHER                            0.167
WHITE                            0.172
Name: mean, dtype: float64

In [72]:
round(1.96 * FP_RaceSex_df['M']["std"] / np.sqrt(5),3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.050
ASIAN                            0.010
BLACK/AFRICAN AMERICAN           0.007
HISPANIC/LATINO                  0.010
OTHER                            0.007
WHITE                            0.007
Name: std, dtype: float64

In [73]:
round(FP_RaceSex_df['F']['mean'],3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.189
ASIAN                            0.301
BLACK/AFRICAN AMERICAN           0.289
HISPANIC/LATINO                  0.356
OTHER                            0.187
WHITE                            0.167
Name: mean, dtype: float64

In [74]:
round(1.96 * FP_RaceSex_df['F']["std"] / np.sqrt(5),3)

race
AMERICAN INDIAN/ALASKA NATIVE    0.026
ASIAN                            0.011
BLACK/AFRICAN AMERICAN           0.008
HISPANIC/LATINO                  0.016
OTHER                            0.008
WHITE                            0.007
Name: std, dtype: float64

In [75]:
factors = ['F', 'M']
race =['AMERICAN INDIAN/ALASKA NATIVE', 'ASIAN', 'BLACK/AFRICAN AMERICAN',
       'HISPANIC/LATINO','OTHER','WHITE']
RaceSex_df = pd.DataFrame(race, columns=["race"])
Num_df =  pd.read_csv("./results77/Num_RaceSex.csv")

def FiveRun(factors,want_df,df,Num):
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])
        
        Num['Num_'+factor] = Num[factor]
    
    Num = Num.drop(factors,axis=1)
    want_df = want_df.merge(Num,left_on="race", right_on="race")
    
    return want_df

want = FiveRun(factors,RaceSex_df,FP_RaceSex_df,Num_df)
want.to_csv('./results/Inter_RaceSex.csv')
want

,race,F,CI_F,M,CI_M,Unnamed: 0,Num_F,Num_M
0,AMERICAN INDIAN/ALASKA NATIVE,0.189,0.026,0.165,0.050,5,18,23
1,ASIAN,0.301,0.011,0.149,0.010,4,182,225
2,BLACK/AFRICAN AMERICAN,0.289,0.008,0.256,0.007,1,1772,1152
3,HISPANIC/LATINO,0.356,0.016,0.206,0.010,2,451,547
4,OTHER,0.187,0.008,0.167,0.007,3,251,472
5,WHITE,0.167,0.007,0.172,0.007,0,4930,6688


In [76]:
want.loc[want['race']=='HISPANIC/LATINO','CI_M']

3    0.01
Name: CI_M, dtype: float64

In [77]:
want[want['race']=='HISPANIC/LATINO']['CI_M'].tolist()[0]

0.01

In [78]:
FP5_RaceSex = pd.read_csv("./results77/FP_FN_RaceSex.csv")
FP4_RaceSex = pd.read_csv("./results19/FP_FN_RaceSex.csv")
FP3_RaceSex = pd.read_csv("./results38/FP_FN_RaceSex.csv")
FP2_RaceSex = pd.read_csv("./results47/FP_FN_RaceSex.csv")
FP1_RaceSex = pd.read_csv("./results31/FP_FN_RaceSex.csv")
FP_IRaceSex = FP1_RaceSex.append([FP2_RaceSex, FP3_RaceSex,FP4_RaceSex, FP5_RaceSex])
FP_RaceSex =FP_IRaceSex.groupby("race")
FP_RaceSex_df =FP_RaceSex.describe()

In [79]:
FP_RaceSex_df.columns

MultiIndex(levels=[['Unnamed: 0', 'FPR_M', 'FNR_M', 'FPR_F', 'FNR_F'], ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [80]:
factors = ['FPR_M','FNR_M','FPR_F','FNR_F']
race = ['AMERICAN INDIAN/ALASKA NATIVE','ASIAN','BLACK/AFRICAN AMERICAN','HISPANIC/LATINO','OTHER','WHITE']

RaceSex_df = pd.DataFrame(race, columns=["race"])

want = FiveRun_FP_NF(factors, RaceSex_df, FP_RaceSex_df)
want.to_csv('./results/Inter_RaceSex_FNR_FPR.csv')
want

,race,FPR_M,CI_FPR_M,FNR_M,CI_FNR_M,FPR_F,CI_FPR_F,FNR_F,CI_FNR_F
0,AMERICAN INDIAN/ALASKA NATIVE,0.165,0.050,0.533,0.065,0.189,0.026,0.000,0.000
1,ASIAN,0.149,0.010,0.181,0.008,0.301,0.011,0.098,0.014
2,BLACK/AFRICAN AMERICAN,0.256,0.007,0.207,0.010,0.289,0.008,0.149,0.009
3,HISPANIC/LATINO,0.206,0.010,0.194,0.009,0.356,0.016,0.141,0.016
4,OTHER,0.167,0.007,0.165,0.014,0.187,0.008,0.259,0.011
5,WHITE,0.172,0.007,0.273,0.007,0.167,0.007,0.231,0.008


In [62]:
print('Number of images in Ins Rac intersection with actual NF=0')
df=pd.read_csv('./results47/Num_RaceSex.csv')

Number of images in Ins Rac intersection with actual NF=0


,Unnamed: 0,race,M,F
0,0,WHITE,6688,4930
1,1,BLACK/AFRICAN AMERICAN,1152,1772
2,2,HISPANIC/LATINO,547,451
3,3,OTHER,472,251
4,4,ASIAN,225,182
5,5,AMERICAN INDIAN/ALASKA NATIVE,23,18


In [64]:
df2=df.drop(['Unnamed: 0'], axis=1)

In [65]:
df2.sum(axis=1)

0    11618
1     2924
2      998
3      723
4      407
5       41
dtype: int64

In [3]:
print('Number of images in Ins Rac intersection with actual NF=1')
pd.read_csv('./results47/Num_RaceSex_NF1.csv')

Number of images in Ins Rac intersection with actual NF=1


,Unnamed: 0,race,M,F
0,0,WHITE,2717,2419
1,1,BLACK/AFRICAN AMERICAN,659,1411
2,2,HISPANIC/LATINO,308,474
3,3,OTHER,220,135
4,4,ASIAN,116,151
5,5,AMERICAN INDIAN/ALASKA NATIVE,6,7


In [66]:
df=pd.read_csv('./results47/Num_RaceSex_NF1.csv')
df2=df.drop(['Unnamed: 0'], axis=1)
df2.sum(axis=1)

0    5136
1    2070
2     782
3     355
4     267
5      13
dtype: int64

### Insurance-Age

In [81]:
FP5_InsAge = pd.read_csv("./results77/FP_InsAge.csv")
FP4_InsAge = pd.read_csv("./results19/FP_InsAge.csv")
FP3_InsAge = pd.read_csv("./results38/FP_InsAge.csv")
FP2_InsAge = pd.read_csv("./results47/FP_InsAge.csv")
FP1_InsAge = pd.read_csv("./results31/FP_InsAge.csv")
FP_InsAge = FP1_InsAge.append([FP2_InsAge, FP3_InsAge,FP4_InsAge, FP5_InsAge])
FP_InsAge =FP_InsAge.groupby("age")
FP_InsAge_df =FP_InsAge.describe()

In [82]:
round(FP_InsAge_df['Medicare']['mean'],3)

age
0-20     0.303
20-40    0.406
40-60    0.238
60-80    0.124
80-      0.098
Name: mean, dtype: float64

In [83]:
round(1.96 * FP_InsAge_df['Medicare']["std"] / np.sqrt(5),3)

age
0-20     0.051
20-40    0.012
40-60    0.008
60-80    0.007
80-      0.008
Name: std, dtype: float64

In [84]:
round(FP_InsAge_df['Other']['mean'],3)

age
0-20     0.341
20-40    0.268
40-60    0.233
60-80    0.148
80-      0.098
Name: mean, dtype: float64

In [85]:
round(1.96 * FP_InsAge_df['Other']["std"] / np.sqrt(5),3)

age
0-20     0.019
20-40    0.012
40-60    0.005
60-80    0.004
80-      0.009
Name: std, dtype: float64

In [86]:
round(FP_InsAge_df['Medicaid']['mean'],3)

age
0-20     0.729
20-40    0.431
40-60    0.258
60-80    0.128
80-      0.012
Name: mean, dtype: float64

In [87]:
round(1.96 * FP_InsAge_df['Medicaid']["std"] / np.sqrt(5),3)

age
0-20     0.052
20-40    0.018
40-60    0.008
60-80    0.011
80-      0.014
Name: std, dtype: float64

In [88]:
factors = ['Medicaid', 'Other','Medicare']
age =['0-20', '20-40', '40-60', '60-80','80-']
AgeIns_df = pd.DataFrame(age, columns=["age"])
Num_df =  pd.read_csv("./results77/Num_InsAge.csv")


def FiveRun(factors,want_df,df,Num):
    for factor in factors:    
        dfM0 = round(df[factor]['mean'],3)
        dfM2 = round(1.96 * df[factor]["std"] / np.sqrt(5),3)
        want_df[factor] = pd.DataFrame(dfM0.values.tolist(),columns =[factor])
        want_df['CI_'+factor] = pd.DataFrame(dfM2.values.tolist(),columns =['CI_'+factor])
        
        Num['Num_'+factor] = Num[factor]
    
    Num = Num.drop(factors,axis=1)
    want_df = want_df.merge(Num,left_on="age", right_on="age")
    
    return want_df

want = FiveRun(factors,AgeIns_df,FP_InsAge_df,Num_df)
want.to_csv('./results/Inter_AgeIns.csv')

In [89]:
want

,age,Medicaid,CI_Medicaid,Other,CI_Other,Medicare,CI_Medicare,Unnamed: 0,Num_Medicaid,Num_Other,Num_Medicare
0,0-20,0.729,0.052,0.341,0.019,0.303,0.051,4,14,85,39
1,20-40,0.431,0.018,0.268,0.012,0.406,0.012,2,359,1447,263
2,40-60,0.258,0.008,0.233,0.005,0.238,0.008,1,920,3153,2378
3,60-80,0.128,0.011,0.148,0.004,0.124,0.007,0,265,2270,4736
4,80-,0.012,0.014,0.098,0.009,0.098,0.008,3,33,410,1091


In [90]:
FP5_InsAge = pd.read_csv("./results77/FP_FN_InsAge.csv")
FP4_InsAge = pd.read_csv("./results19/FP_FN_InsAge.csv")
FP3_InsAge = pd.read_csv("./results38/FP_FN_InsAge.csv")
FP2_InsAge = pd.read_csv("./results47/FP_FN_InsAge.csv")
FP1_InsAge = pd.read_csv("./results31/FP_FN_InsAge.csv")
FP_InsAge = FP1_InsAge.append([FP2_InsAge, FP3_InsAge,FP4_InsAge, FP5_InsAge])
FP_InsAge =FP_InsAge.groupby("age")
FP_InsAge_df =FP_InsAge.describe()

In [91]:
FP_InsAge_df.columns

MultiIndex(levels=[['Unnamed: 0', 'FPR_Medicare', 'FNR_Medicare', 'FPR_Other', 'FNR_Other', 'FPR_Medicaid', 'FNR_Medicaid'], ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [92]:
factors = ['FPR_Medicare','FNR_Medicare','FPR_Other','FNR_Other', 'FPR_Medicaid', 'FNR_Medicaid']
age = ['0-20','20-40','40-60','60-80','80-']

AgeIns_df = pd.DataFrame(age, columns=["age"])

want = FiveRun_FP_NF(factors, AgeIns_df, FP_InsAge_df)
want.to_csv('./results/Inter_AgeIns_FNR_FPR.csv')
want

,age,FPR_Medicare,CI_FPR_Medicare,FNR_Medicare,CI_FNR_Medicare,FPR_Other,CI_FPR_Other,FNR_Other,CI_FNR_Other,FPR_Medicaid,CI_FPR_Medicaid,FNR_Medicaid,CI_FNR_Medicaid
0,0-20,0.303,0.051,0.178,0.087,0.341,0.019,0.043,0.009,0.729,0.052,0.014,0.016
1,20-40,0.406,0.012,0.054,0.006,0.268,0.012,0.101,0.004,0.431,0.018,0.075,0.008
2,40-60,0.238,0.008,0.231,0.005,0.233,0.005,0.173,0.005,0.258,0.008,0.235,0.016
3,60-80,0.124,0.007,0.411,0.014,0.148,0.004,0.321,0.014,0.128,0.011,0.314,0.024
4,80-,0.098,0.008,0.486,0.017,0.098,0.009,0.468,0.038,0.012,0.014,1.000,0.000


In [67]:
print('Number of images in Ins Age intersection with actual NF=0')
pd.read_csv('./results47/Num_InsAge.csv')

Number of images in Ins Age intersection with actual NF=0


,Unnamed: 0,age,Medicare,Other,Medicaid
0,0,60-80,4736,2270,265
1,1,40-60,2378,3153,920
2,2,20-40,263,1447,359
3,3,80-,1091,410,33
4,4,0-20,39,85,14


In [68]:
df=pd.read_csv('./results47/Num_InsAge.csv')
df2=df.drop(['Unnamed: 0'], axis=1)
df2.sum(axis=0)

age         60-8040-6020-4080-0-20
Medicare                      8507
Other                         7365
Medicaid                      1591
dtype: object

In [58]:
print('Number of images in Ins Age intersection with actual NF=1')
pd.read_csv('./results47/Num_InsAge_NF1.csv')



Number of images in Ins Age intersection with actual NF=1


,Unnamed: 0,age,Medicare,Other,Medicaid
0,0,60-80,1339,825,93
1,1,40-60,1232,1947,495
2,2,20-40,312,1468,496
3,3,80-,238,103,2
4,4,0-20,9,194,29


In [69]:
df=pd.read_csv('./results47/Num_InsAge_NF1.csv')
df2=df.drop(['Unnamed: 0'], axis=1)
df2.sum(axis=0)



age         60-8040-6020-4080-0-20
Medicare                      3130
Other                         4537
Medicaid                      1115
dtype: object

### Race-Age

In [93]:
FP5_RaceAge = pd.read_csv("./results77/FP_RaceAge.csv")
FP4_RaceAge = pd.read_csv("./results19/FP_RaceAge.csv")
FP3_RaceAge = pd.read_csv("./results38/FP_RaceAge.csv")
FP2_RaceAge = pd.read_csv("./results47/FP_RaceAge.csv")
FP1_RaceAge = pd.read_csv("./results31/FP_RaceAge.csv")
FP_RaceAge = FP1_RaceAge.append([FP2_RaceAge, FP3_RaceAge,FP4_RaceAge, FP5_RaceAge])
FP_RaceAge = FP_RaceAge.groupby("age")
FP_RaceAge_df =FP_RaceAge.describe()

In [94]:
factors = ['White', 'Black','Hisp','Other','Asian','American']
age =['0-20', '20-40', '40-60', '60-80','80-']
RaceAge_df = pd.DataFrame(age, columns=["age"])
Num_df =  pd.read_csv("./results77/Num_RaceAge.csv")





want = FiveRun(factors,RaceAge_df,FP_RaceAge_df,Num_df)
want.to_csv('./results/Inter_RaceAge.csv')
want

,age,White,CI_White,Black,CI_Black,Hisp,CI_Hisp,Other,CI_Other,Asian,CI_Asian,American,CI_American,Unnamed: 0,Num_White,Num_Black,Num_Hisp,Num_Other,Num_Asian,Num_American
0,0-20,0.302,0.027,0.737,0.033,0.200,0.240,1.000,0.000,0.000,0.000,NaN,NaN,4,108,19,2,4,2,0
1,20-40,0.331,0.014,0.346,0.011,0.231,0.012,0.300,0.024,0.269,0.018,0.200,0.044,2,930,617,315,70,64,9
2,40-60,0.213,0.006,0.308,0.009,0.342,0.014,0.222,0.007,0.322,0.012,1.000,0.000,1,4278,1225,399,230,115,2
3,60-80,0.117,0.007,0.200,0.005,0.232,0.014,0.123,0.009,0.200,0.005,0.113,0.033,0,5143,942,268,364,160,30
4,80-,0.102,0.009,0.117,0.012,0.143,0.000,0.095,0.013,0.030,0.009,NaN,NaN,3,1159,121,14,55,66,0


In [95]:
want.loc[want['age']=='0-20','Hisp']

0    0.2
Name: Hisp, dtype: float64

In [96]:
FP5_RaceAge = pd.read_csv("./results77/FP_FN_RaceAge.csv")
FP4_RaceAge = pd.read_csv("./results19/FP_FN_RaceAge.csv")
FP3_RaceAge = pd.read_csv("./results38/FP_FN_RaceAge.csv")
FP2_RaceAge = pd.read_csv("./results47/FP_FN_RaceAge.csv")
FP1_RaceAge = pd.read_csv("./results31/FP_FN_RaceAge.csv")
FP_RaceAge = FP1_RaceAge.append([FP2_RaceAge, FP3_RaceAge,FP4_RaceAge, FP5_RaceAge])
FP_RaceAge = FP_RaceAge.groupby("age")
FP_RaceAge_df =FP_RaceAge.describe()

In [97]:
FP_RaceAge_df.columns

MultiIndex(levels=[['Unnamed: 0', 'FPR_White', 'FNR_White', 'FPR_Black', 'FNR_Black', 'FPR_Hisp', 'FNR_Hisp', 'FPR_Other', 'FNR_Other', 'FPR_Asian', 'FNR_Asian', 'FPR_American', 'FNR_American'], ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [98]:
factors = ['FPR_White', 'FNR_White', 'FPR_Black', 'FNR_Black', 'FPR_Hisp', 'FNR_Hisp', 'FPR_Other', 'FNR_Other', 'FPR_Asian', 'FNR_Asian', 'FPR_American', 'FNR_American']
age =['0-20', '20-40', '40-60', '60-80','80-']

AgeRace_df = pd.DataFrame(age, columns=["age"])

want = FiveRun_FP_NF(factors, AgeRace_df, FP_RaceAge_df)
want.to_csv('./results/Inter_AgeRace_FNR_FPR.csv')
want


,age,FPR_White,CI_FPR_White,FNR_White,CI_FNR_White,FPR_Black,CI_FPR_Black,FNR_Black,CI_FNR_Black,FPR_Hisp,...,FNR_Other,CI_FNR_Other,FPR_Asian,CI_FPR_Asian,FNR_Asian,CI_FNR_Asian,FPR_American,CI_FPR_American,FNR_American,CI_FNR_American
0,0-20,0.302,0.027,0.063,0.014,0.737,0.033,0.000,0.000,0.200,...,0.027,0.032,0.000,0.000,0.042,0.039,NaN,NaN,NaN,NaN
1,20-40,0.331,0.014,0.084,0.007,0.346,0.011,0.071,0.003,0.231,...,0.119,0.019,0.269,0.018,0.180,0.018,0.200,0.044,0.2,0.065
2,40-60,0.213,0.006,0.220,0.005,0.308,0.009,0.177,0.009,0.342,...,0.175,0.019,0.322,0.012,0.082,0.007,1.000,0.000,0.0,0.000
3,60-80,0.117,0.007,0.394,0.014,0.200,0.005,0.357,0.020,0.232,...,0.301,0.016,0.200,0.005,0.191,0.007,0.113,0.033,1.0,0.000
4,80-,0.102,0.009,0.492,0.016,0.117,0.012,0.382,0.021,0.143,...,NaN,NaN,0.030,0.009,0.100,0.196,NaN,NaN,NaN,NaN


In [49]:
print('Number of images in Race Age intersection with actual NF=0')
df=pd.read_csv('./results47/Num_RaceAge.csv')
df2=df.drop(['Unnamed: 0'], axis=1)
df2

Number of images in Race Age intersection with actual NF=0


,age,White,Black,Hisp,Other,Asian,American
0,60-80,5143,942,268,364,160,30
1,40-60,4278,1225,399,230,115,2
2,20-40,930,617,315,70,64,9
3,80-,1159,121,14,55,66,0
4,0-20,108,19,2,4,2,0


In [50]:
df2.sum(axis=1)

0    6907
1    6249
2    2005
3    1415
4     135
dtype: int64

In [47]:
print('Number of images in Ins Age intersection with actual NF=1')
df=pd.read_csv('./results47/Num_RaceAge_NF1.csv')
df2=df.drop(['Unnamed: 0'], axis=1)
df2


Number of images in Ins Age intersection with actual NF=1


,age,White,Black,Hisp,Other,Asian,American
0,60-80,1532,343,118,119,89,2
1,40-60,2156,853,347,157,107,5
2,20-40,1043,783,306,64,50,6
3,80-,260,45,7,0,2,0
4,0-20,145,46,4,15,19,0


In [48]:
df2.sum(axis=1)



0    2203
1    3625
2    2252
3     314
4     229
dtype: int64

In [39]:
2156+853+347+157+107


3620

## AUC Performance confidence interval 

In [99]:
Eval5 = pd.read_csv("./results77/Eval.csv")
Eval4 = pd.read_csv("./results19/Eval.csv")
Eval3 = pd.read_csv("./results38/Eval.csv")
Eval2 = pd.read_csv("./results47/Eval.csv")
Eval1 = pd.read_csv("./results31/Eval.csv")

In [100]:

Evalall = Eval1[['label','auc']]

Evalall["auc2"]=Eval2['auc']
Evalall["auc3"]=Eval3['auc']
Evalall["auc4"]=Eval4['auc']
Evalall["auc5"]=Eval5['auc']
Evalall

,label,auc,auc2,auc3,auc4,auc5
0,Airspace Opacity,0.784405,0.783839,0.781926,0.780642,0.780812
1,Atelectasis,0.837631,0.837549,0.836750,0.835749,0.835974
2,Cardiomegaly,0.829379,0.829341,0.826660,0.827601,0.824610
3,Consolidation,0.844549,0.845112,0.843367,0.843544,0.842740
4,Edema,0.905035,0.905858,0.904471,0.903695,0.900561
5,Enlarged Cardiomediastinum,0.760054,0.752302,0.759163,0.753896,0.760663
6,Fracture,0.719671,0.707227,0.717175,0.729794,0.718162
7,Lung Lesion,0.770842,0.779997,0.777454,0.767263,0.764300
8,No Finding,0.870459,0.869439,0.868266,0.868240,0.865947
9,Pleural Effusion,0.934543,0.933334,0.933444,0.932899,0.932870


In [101]:
Evala_dis =Evalall.groupby("label")

In [102]:
round(Evalall.mean(axis=0),3)

auc     0.835
auc2    0.834
auc3    0.833
auc4    0.833
auc5    0.832
dtype: float64

In [103]:
print("MIMIC Mean of 14 aucs mean over 5 run:     ",round(Evalall.mean(axis=0).mean(),3))
print("MIMIC Confidence interval of 14 aucs mean over 5 run:   ",round(1.96 * Evalall.mean(axis=0).std()/np.sqrt(5) ,3 ) )

MIMIC Mean of 14 aucs mean over 5 run:      0.834
MIMIC Confidence interval of 14 aucs mean over 5 run:    0.001


In [104]:
print("MIMIC Mean of auce per disease over 5 run:     ")
round(Evalall.mean(axis=1),3)

MIMIC Mean of auce per disease over 5 run:     


0     0.782
1     0.837
2     0.828
3     0.844
4     0.904
5     0.757
6     0.718
7     0.772
8     0.868
9     0.933
10    0.848
11    0.748
12    0.903
13    0.927
dtype: float64

In [105]:
print("MIMIC Confidence interval of auce per disease over 5 run:     ")
round(1.96 * Evalall.std(axis=1)/ np.sqrt(5),3)

MIMIC Confidence interval of auce per disease over 5 run:     


0     0.002
1     0.001
2     0.002
3     0.001
4     0.002
5     0.003
6     0.007
7     0.006
8     0.001
9     0.001
10    0.003
11    0.005
12    0.002
13    0.001
dtype: float64